# LightGlue Demo
In this notebook we match two pairs of images using LightGlue with early stopping and point pruning.

In [1]:
# If we are on colab: this clones the repo and installs the dependencies
from pathlib import Path

if Path.cwd().name != "LightGlue":
    !git clone --quiet https://github.com/cvg/LightGlue/
    %cd LightGlue
    !pip install --progress-bar off --quiet -e .

from lightglue import LightGlue, SuperPoint, DISK
from lightglue.utils import load_image, rbd
from lightglue import viz2d
import torch

torch.set_grad_enabled(False)
images = Path("assets")


## test the LightGlue class  dd

fatal: destination path 'LightGlue' already exists and is not an empty directory.
/Users/benmartin/Library/CloudStorage/GoogleDrive-btmarti25@gmail.com/My Drive/Projects/stereo_matching_lightglue/LightGlue


## Load extractor and matcher module
In this example we use SuperPoint features combined with LightGlue.

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 'mps', 'cpu'

extractor = SuperPoint(max_num_keypoints=2048).eval().to(device)  # load the extractor
matcher = LightGlue(features="superpoint").eval().to(device)

In [3]:
%pip install ultralytics
from IPython import display
display.clear_output()

from ultralytics import YOLO

In [26]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
from sync_tools import calculate_offset_fft
from ultralytics import YOLO
from track_and_match_tools import get_yolo_tracks_left_right, get_id_matches_lightglue

model = YOLO("/Users/benmartin/Library/CloudStorage/GoogleDrive-btmarti25@gmail.com/My Drive/best_close_chromis.pt")
video1_path = "/Volumes/HD_Feeding/30_11_2023/S1_L1/Card_1_left/DCIM/100GOPRO/GX030035.MP4"
video2_path = "/Volumes/HD_Feeding/30_11_2023/S1_L1/Card_2_right/DCIM/100GOPRO/GX030455.MP4"

#get freames per second of video1
import cv2
cap = cv2.VideoCapture(video1_path)
fps = cap.get(cv2.CAP_PROP_FPS)

#offset = int(fps*calculate_offset_fft(video1_path, video2_path))
offset = 61
start_frame = 100

n_frames = 2
left_tracks, right_tracks = get_yolo_tracks_left_right(start_frame, n_frames, offset, model, video1_path, video2_path)
all_matched_ids = get_id_matches_lightglue(start_frame, offset, n_frames, video1_path, video2_path,left_tracks, right_tracks)

print("Left Tracks:", left_tracks)
print("Right Tracks:", right_tracks)


0: 1088x1920 6 chromiss, 1837.6ms
Speed: 26.2ms preprocess, 1837.6ms inference, 4.1ms postprocess per image at shape (1, 3, 1088, 1920)

0: 1088x1920 6 chromiss, 1680.9ms
Speed: 13.7ms preprocess, 1680.9ms inference, 2.8ms postprocess per image at shape (1, 3, 1088, 1920)

0: 1088x1920 7 chromiss, 1709.1ms
Speed: 12.8ms preprocess, 1709.1ms inference, 1.4ms postprocess per image at shape (1, 3, 1088, 1920)

0: 1088x1920 6 chromiss, 1691.1ms
Speed: 11.5ms preprocess, 1691.1ms inference, 1.4ms postprocess per image at shape (1, 3, 1088, 1920)


Left Tracks: [[          1           0      2401.7      692.83      2542.7      789.43]
 [          2           0        3141      161.02      3261.7      231.69]
 [          3           0      3357.6      452.89      3513.1       513.6]
 [          4           0      1863.2      887.94      1930.4      943.71]
 [          5           0      2091.7      1376.9      2249.7      1442.7]
 [          6           0        1951      607.78      1987.7      646.18]
 [          1           1        2408       698.4        2548      792.16]
 [          2           1      3146.1      162.45      3263.2      234.33]
 [          3           1      3361.1      452.84      3520.1      514.39]
 [          4           1      1864.4      893.04        1933      946.64]
 [          5           1      2098.4      1376.1      2261.5      1445.6]
 [          6           1      1948.4      608.45      1989.6      645.09]]
Right Tracks: [[    0.89709           0      1204.5      507.44      1344.2      603.5

In [12]:
n_frames = 2
left_tracks, right_tracks = get_yolo_tracks_left_right(start_frame, n_frames, offset, model, video1_path, video2_path)
all_matched_ids = get_id_matches_lightglue(start_frame, offset, n_frames, video1_path, video2_path,left_tracks, right_tracks)

print("Left Tracks:", left_tracks)
print("Right Tracks:", right_tracks)



0: 1088x1920 6 chromiss, 1830.9ms
Speed: 19.0ms preprocess, 1830.9ms inference, 3.6ms postprocess per image at shape (1, 3, 1088, 1920)

0: 1088x1920 6 chromiss, 1754.5ms
Speed: 13.2ms preprocess, 1754.5ms inference, 2.8ms postprocess per image at shape (1, 3, 1088, 1920)

0: 1088x1920 1 chromis, 1730.3ms
Speed: 14.1ms preprocess, 1730.3ms inference, 1.3ms postprocess per image at shape (1, 3, 1088, 1920)

0: 1088x1920 6 chromiss, 1763.3ms
Speed: 11.9ms preprocess, 1763.3ms inference, 0.9ms postprocess per image at shape (1, 3, 1088, 1920)


NameError: name 'load_image' is not defined

In [20]:
import numpy as np

def find_matches(left_bbs, right_bbs, m_kpts0, m_kpts1):
    matches_in_box_left = []
    matches_in_box_left_index = []

    for bb in left_bbs:
        id, frame, x1, y1, x2, y2 = bb
        for j, match in enumerate(m_kpts0):
            if x1 < match[0] < x2 and y1 < match[1] < y2:
                matches_in_box_left.append(id)
                matches_in_box_left_index.append(j)

    matches_in_box_right = []
    matches_in_box_right_index = []

    for bb in right_bbs:
        id, frame, x1, y1, x2, y2 = bb
        for j, match in enumerate(m_kpts1):
            if x1 < match[0] < x2 and y1 < match[1] < y2:
                matches_in_box_right.append(id)
                matches_in_box_right_index.append(j)

    # Find common elements
    common_elements = set(matches_in_box_left_index).intersection(set(matches_in_box_right_index))

    # Create a table to store the indices
    table = []

    # Find the indices of common elements in both lists
    for element in common_elements:
        indices_list1 = [i for i, x in enumerate(matches_in_box_left_index) if x == element]
        indices_list2 = [i for i, x in enumerate(matches_in_box_right_index) if x == element]
        for index1 in indices_list1:
            for index2 in indices_list2:
                table.append({'Value': element, 'Index in list1': index1, 'Index in list2': index2})

    # Extract indices into NumPy arrays
    left_ids = np.array([row['Index in list1'] for row in table], dtype=int)
    right_ids = np.array([row['Index in list2'] for row in table], dtype=int)

    # Convert lists to NumPy arrays
    matches_in_box_right = np.array(matches_in_box_right)
    matches_in_box_left = np.array(matches_in_box_left)

    # Use a set to store unique pairs
    unique_matches = set((matches_in_box_left[left_id], matches_in_box_right[right_id])
                         for left_id, right_id in zip(left_ids, right_ids))

    # Convert the set back to a list
    matched_data = list(unique_matches)

    return matched_data

# Example usage:


matched_data = find_matches(left_bbs, right_bbs, m_kpts0, m_kpts1)
print(matched_data)


[(6.0, 12.0), (4.0, 10.0), (1.0, 7.0), (3.0, 9.0)]


## Easy example
The top image shows the matches, while the bottom image shows the point pruning across layers. In this case, LightGlue prunes a few points with occlusions, but is able to stop the context aggregation after 4/9 layers.

In [41]:
import cv2
import numpy as np

def get_yolo_tracks_left_right(start_frame, n_frames, offset, model, video1_path, video2_path):
    # Initialize video capture
    cap1 = cv2.VideoCapture(video1_path)
    cap1.set(cv2.CAP_PROP_POS_FRAMES, start_frame + offset)
    
    cap2 = cv2.VideoCapture(video2_path)
    cap2.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    left_tracks = []
    right_tracks = []

    # Process left video
    for i in range(n_frames):
        ret, frame = cap1.read()
        if not ret:
            break
        results_left = model.track(source=frame, persist=True)
        
        if results_left[0].boxes is not None:
            for box in results_left[0].boxes.data:
                x1, y1, x2, y2, id, conf = box[0:6].detach().cpu().numpy()
                frame_number = i
                left_tracks.append([id, frame_number, x1, y1, x2, y2])

    # Process right video
    for i in range(n_frames):
        ret, frame = cap2.read()
        if not ret:
            break
        results_right = model.track(source=frame, persist=True)
        
        if results_right[0].boxes is not None:
            for box in results_right[0].boxes.data:
                x1, y1, x2, y2, id, conf = box[0:6].detach().cpu().numpy()
                frame_number = i
                right_tracks.append([id, frame_number, x1, y1, x2, y2])

    # Convert lists to numpy arrays
    left_tracks = np.array(left_tracks)
    right_tracks = np.array(right_tracks)

    return left_tracks, right_tracks



In [49]:
import cv2
import numpy as np

def get_id_matches_lightglue(start_frame, offset, n_frames, video1_path, video2_path,left_tracks, right_tracks):
    # Initialize video capture
    cap1 = cv2.VideoCapture(video1_path)
    cap1.set(cv2.CAP_PROP_POS_FRAMES, offset + start_frame)
    
    cap2 = cv2.VideoCapture(video2_path)
    cap2.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    
    # Assume these are provided by your environment
    # rt = np.array(right_tracks)  # Right bounding boxes with tracking data
    # lt = np.array(left_tracks)   # Left bounding boxes with tracking data

    # You would populate rt and lt with your tracking data before this function call
    rt = np.array(right_tracks)  # Replace with actual right tracking data
    lt = np.array(left_tracks)   # Replace with actual left tracking data

    all_matched_ids = []

    for i in range(n_frames):
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()
        if not ret1 or not ret2:
            break

        # Save the frames temporarily
        cv2.imwrite("frame1.png", cv2.cvtColor(frame1, cv2.COLOR_RGB2BGR))
        cv2.imwrite("frame2.png", cv2.cvtColor(frame2, cv2.COLOR_RGB2BGR))
        image0 = load_image("frame1.png")
        image1 = load_image("frame2.png")

        feats0 = extractor.extract(image0.to(device))
        feats1 = extractor.extract(image1.to(device))
        matches01 = matcher({"image0": feats0, "image1": feats1})
        feats0, feats1, matches01 = [
            rbd(x) for x in [feats0, feats1, matches01]
        ]  # remove batch dimension

        kpts0, kpts1, matches = feats0["keypoints"], feats1["keypoints"], matches01["matches"]
        m_kpts0, m_kpts1 = kpts0[matches[..., 0]], kpts1[matches[..., 1]]

        # Get bounding boxes for the current frame
        left_bbs = lt[lt[:, 1] == i, :]
        right_bbs = rt[rt[:, 1] == i, :]

        matched_ids = find_matches(left_bbs, right_bbs, m_kpts0, m_kpts1)
        
        # Add the frame number to each match
        matched_ids_with_frame = [(i, match[0], match[1]) for match in matched_ids]

        # Append the matches to the data structure
        all_matched_ids.extend(matched_ids_with_frame)

    # Convert to numpy array
    all_matched_ids = np.array(all_matched_ids)
    
    # Remove rows with non-integer values
    all_matched_ids = all_matched_ids[
        np.all(all_matched_ids[:, 1:].astype(float) == all_matched_ids[:, 1:].astype(int), axis=1)
    ]
    
    return np.array(all_matched_ids)



In [45]:
# Example usage

start_frame = 100
offset = 61
n_frames = 20
left_tracks, right_tracks = get_yolo_tracks_left_right(start_frame, n_frames, offset, model, video1_path, video2_path)
all_matched_ids = get_id_matches_lightglue(start_frame, offset, n_frames, video1_path, video2_path,left_tracks, right_tracks)

print("Left Tracks:", left_tracks)
print("Right Tracks:", right_tracks)



0: 1088x1920 6 chromiss, 1663.8ms
Speed: 15.7ms preprocess, 1663.8ms inference, 3.5ms postprocess per image at shape (1, 3, 1088, 1920)

0: 1088x1920 6 chromiss, 1643.8ms
Speed: 13.5ms preprocess, 1643.8ms inference, 3.2ms postprocess per image at shape (1, 3, 1088, 1920)

0: 1088x1920 6 chromiss, 1648.7ms
Speed: 11.8ms preprocess, 1648.7ms inference, 4.1ms postprocess per image at shape (1, 3, 1088, 1920)

0: 1088x1920 6 chromiss, 1724.1ms
Speed: 12.8ms preprocess, 1724.1ms inference, 3.3ms postprocess per image at shape (1, 3, 1088, 1920)

0: 1088x1920 6 chromiss, 1752.4ms
Speed: 12.1ms preprocess, 1752.4ms inference, 1.2ms postprocess per image at shape (1, 3, 1088, 1920)

0: 1088x1920 6 chromiss, 1832.4ms
Speed: 13.2ms preprocess, 1832.4ms inference, 3.3ms postprocess per image at shape (1, 3, 1088, 1920)

0: 1088x1920 6 chromiss, 1780.2ms
Speed: 14.8ms preprocess, 1780.2ms inference, 2.5ms postprocess per image at shape (1, 3, 1088, 1920)

0: 1088x1920 6 chromiss, 1783.4ms
Speed:

NameError: name 'find_unique_matches' is not defined

In [50]:
all_matched_ids = get_id_matches_lightglue(start_frame, offset, n_frames, video1_path, video2_path,left_tracks, right_tracks)


In [51]:
print(all_matched_ids)

[[          1          44          51]
 [          1          42          49]
 [          1          45          52]
 [          2          44          51]
 [          2          45          52]
 [          2          42          49]
 [          2          47          54]
 [          3          44          51]
 [          3          45          52]
 [          3          47          54]
 [          4          44          51]
 [          4          45          52]
 [          4          47          54]
 [          5          44          51]
 [          5          45          52]
 [          5          47          54]
 [          6          44          51]
 [          6          47          54]
 [          7          44          51]
 [          7          47          54]
 [          8          44          51]
 [          8          47          54]
 [          9          44          51]
 [          9          47          54]
 [         10          44          51]
 [         10          47

In [34]:
import numpy as np


cap1 = cv2.VideoCapture(video1_path)
cap1.set(cv2.CAP_PROP_POS_FRAMES, offset+ start_frame)

cap2 = cv2.VideoCapture(video2_path)
cap2.set(cv2.CAP_PROP_POS_FRAMES, 0 + start_frame)

rt = np.array(right_tracks)
lt = np.array(left_tracks)

all_matched_ids = []
# Assuming cap1 and cap2 are your video capture objects
for i in range(n_frames):
    ret, frame1 = cap1.read()
    ret, frame2 = cap2.read()

    # Save the frames
    cv2.imwrite("frame1.png", cv2.cvtColor(frame1, cv2.COLOR_RGB2BGR))
    cv2.imwrite("frame2.png", cv2.cvtColor(frame2, cv2.COLOR_RGB2BGR))
    image0 = load_image("frame1.png")
    image1 = load_image("frame2.png")

    feats0 = extractor.extract(image0.to(device))
    feats1 = extractor.extract(image1.to(device))
    matches01 = matcher({"image0": feats0, "image1": feats1})
    feats0, feats1, matches01 = [
        rbd(x) for x in [feats0, feats1, matches01]
    ]  # remove batch dimension

    kpts0, kpts1, matches = feats0["keypoints"], feats1["keypoints"], matches01["matches"]
    m_kpts0, m_kpts1 = kpts0[matches[..., 0]], kpts1[matches[..., 1]]

    # Get bounding boxes for the current frame
    left_bbs = lt[lt[:, 1] == i, :]
    right_bbs = rt[rt[:, 1] == i, :]

    matched_ids = find_matches(left_bbs, right_bbs, m_kpts0, m_kpts1)
    
    # Add the frame number to each match
    matched_ids_with_frame = [(i, match[0], match[1]) for match in matched_ids]

    # Append the matches to the data structure
    all_matched_ids.extend(matched_ids_with_frame)

# Print or process the matched IDs for all frames
print(np.array(all_matched_ids))

    


    
    

[[          0           1     0.89709]
 [          0           4     0.88638]
 [          0           6     0.85665]
 [          0           3     0.88909]
 [          1           4          11]
 [          1           1           8]
 [          1           3          10]
 [          2           4          11]
 [          2           6          13]
 [          2           1           8]
 [          2           3          10]
 [          3           4          11]
 [          3           6          13]
 [          3           3          10]
 [          4           4          11]
 [          4           6          13]
 [          4           3          10]
 [          5           4          11]
 [          5           6          13]
 [          5           3          10]
 [          6           6          13]
 [          6           3          10]
 [          7           6          13]
 [          7           3          10]
 [          8           6          13]
 [          8           3